#Settings

In [1]:
#Drive Mount
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install ultralytics opencv-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.5 MB/s eta 0:00:00


In [ ]:
#Import libraries
import os
import yaml
import json
import logging
import shutil
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional
from datetime import datetime
from ultralytics import YOLO
import torch
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
#Logger Setting
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger('YOLO')

#GPU Setting
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info(f"Using device: {device}")
if device == 'cuda':
    logger.info(f"GPU: {torch.cuda.get_device_name(0)}")
    logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
#YOLO Setting
@dataclass
class YOLOConfig:
    data_yaml: str = "/content/drive/MyDrive/yolo_dataset/data.yaml"
    output_dir: str = "/content/drive/MyDrive/YOLO/Outputs_11 18 ver"

    #model
    model_name: str = "yolo12n.pt"

    #Training
    epochs: int = 200
    batch_size: int = 8
    img_size: int = 960
    patience: int = 30

    #Optimizer
    optimizer: str = "AdamW"
    lr0: float = 0.005
    lrf: float = 0.01
    momentum: float = 0.937
    weight_decay: float = 0.0005
    warmup_epochs: int = 3
    warmup_momentum: float = 0.8
    warmup_bias_lr: float = 0.1
    box: float = 8.5
    cls: float = 0.5

    # Data Augmentation
    hsv_h: float = 0.02
    hsv_s: float = 0.8
    hsv_v: float = 0.5
    degrees: float = 15.0
    translate: float = 0.15
    scale: float = 0.6
    shear: float = 2.0
    perspective: float = 0.0001
    flipud: float = 0.5
    fliplr: float = 0.5
    mosaic: float = 1.0
    mixup: float = 0.15
    copy_paste: float = 0.1

    workers: int = 2
    device: str = '0'
    exist_ok: bool = True
    pretrained: bool = True
    verbose: bool = True
    save: bool = True
    save_period: int = -1
    cache: bool = False
    val: bool = True
    plots: bool = True
    cos_lr: bool = True
    close_mosaic: int = 10

config = YOLOConfig()

Path(config.output_dir).mkdir(parents=True, exist_ok=True)

logger.info("Complete Configuration")
logger.info(f"Output Directory: {config.output_dir}")
logger.info(f"Epochs: {config.epochs}")
logger.info(f"Batch size: {config.batch_size}")
logger.info(f"Optimizer: {config.optimizer}")
logger.info(f"Learning rate: {config.lr0} → {config.lrf}")

In [ ]:
class YOLOTrainer:
    def __init__(self, config: YOLOConfig):
        self.config = config
        self.model = None
        self.results = None
        self.best_weights_path = None

    def initialize_model(self):
        logger.info(f"Load Model: {self.config.model_name}")
        self.model = YOLO(self.config.model_name)

    def train(self):
        if self.model is None:
            self.initialize_model()

        logger.info("Start training")

        # Training parameters
        train_args = {
            'data': self.config.data_yaml,
            'epochs': self.config.epochs,
            'batch': self.config.batch_size,
            'imgsz': self.config.img_size,
            'patience': self.config.patience,
            'device': self.config.device,
            'workers': self.config.workers,
            'project': self.config.output_dir,
            'name': 'train',
            'exist_ok': self.config.exist_ok,
            'pretrained': self.config.pretrained,
            'verbose': self.config.verbose,
            'save': self.config.save,
            'save_period': self.config.save_period,
            'cache': self.config.cache,
            'val': self.config.val,
            'plots': self.config.plots,

            # Optimizer settings
            'optimizer': self.config.optimizer,
            'lr0': self.config.lr0,
            'lrf': self.config.lrf,
            'box': self.config.box,
            'cls': self.config.cls,
            'momentum': self.config.momentum,
            'weight_decay': self.config.weight_decay,
            'warmup_epochs': self.config.warmup_epochs,
            'warmup_momentum': self.config.warmup_momentum,
            'warmup_bias_lr': self.config.warmup_bias_lr,
            'cos_lr': self.config.cos_lr,

            # Data Augmentation
            'hsv_h': self.config.hsv_h,
            'hsv_s': self.config.hsv_s,
            'hsv_v': self.config.hsv_v,
            'degrees': self.config.degrees,
            'translate': self.config.translate,
            'scale': self.config.scale,
            'shear': self.config.shear,
            'perspective': self.config.perspective,
            'flipud': self.config.flipud,
            'fliplr': self.config.fliplr,
            'mosaic': self.config.mosaic,
            'mixup': self.config.mixup,
            'copy_paste': self.config.copy_paste,
            'close_mosaic': self.config.close_mosaic,
        }

        # Training
        self.results = self.model.train(**train_args)

        # Save Best weights
        self.best_weights_path = Path(self.config.output_dir) / 'train' / 'weights' / 'best.pt'
        logger.info(f"Training Completed!")
        logger.info(f"Best weights: {self.best_weights_path}")

        return self.results

# Run
trainer = YOLOTrainer(config)
results = trainer.train()

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=8.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/yolo_dataset/data.yaml, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=30, perspective=0.0001, 

In [ ]:
# Validation
logger.info("Best 모델 검증 시작...")
best_model = YOLO(trainer.best_weights_path)
metrics = best_model.val(data=config.data_yaml)

# Extract metrics
results_dict = {
    "mAP50-95": float(metrics.box.map),
    "mAP50": float(metrics.box.map50),
    "mAP75": float(metrics.box.map75),
    "Precision": float(metrics.box.mp),
    "Recall": float(metrics.box.mr),
}

logger.info("Validation Completed")
logger.info("📈 Metrics")
for key, value in results_dict.items():
    logger.info(f"  {key:15s}: {value:.4f}")

print("\n" + "="*60)
print("Model Performance")
print("="*60)
for key, value in results_dict.items():
    print(f"{key:15s}: {value:.4f}")
print("="*60)

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 17.9±39.2 ms, read: 14.8±15.9 MB/s, size: 39.9 KB)
val: Scanning /content/drive/MyDrive/yolo_dataset/labels/valid.cache... 299 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 299/299 332.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 2.7it/s 6.9s
                   all        299       2766      0.919       0.89      0.953      0.604
Speed: 2.3ms preprocess, 13.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to /content/runs/detect/val

Model Performance
mAP50-95       : 0.6044
mAP50          : 0.9535
mAP75          : 0.7125
Precision      : 0.9192
Recall         : 0.8901


In [ ]:
# Test_image_dir
test_images_dir = "/content/drive/MyDrive/yolo_dataset/images/test"

logger.info(f"Testing: {test_images_dir}")

# Test
predictions = best_model.predict(
    source=str(test_images_dir),
    conf=0.25,
    iou=0.7,
    save=True,
    save_txt=True,
    save_conf=True,
    project=config.output_dir,
    name="predictions",
    exist_ok=True
)

logger.info(f"Test Completd Total {len(predictions)}images")
logger.info(f"Results saved at: {config.output_dir}/predictions")


image 1/119 /content/drive/MyDrive/yolo_dataset/images/test/1000_F_420127490_zYfufzLT1jVxPzLVJzIh46ZI6FSeML3B_jpg.rf.472f2e308640e289e705c6b571703809.jpg: 640x960 16 Pills, 85.9ms
image 2/119 /content/drive/MyDrive/yolo_dataset/images/test/1003948616_jpg.rf.808a0100ee53070b7d8e810fb8322228.jpg: 960x960 18 Pills, 15.0ms
image 3/119 /content/drive/MyDrive/yolo_dataset/images/test/1068691880_jpg.rf.a2a334e22b766b29bfd59e08cc63f4f8.jpg: 960x960 8 Pills, 14.6ms
image 4/119 /content/drive/MyDrive/yolo_dataset/images/test/107091458_jpg.rf.10e3d01898813aa0dbd360d3c45682a3.jpg: 448x960 127 Pills, 85.6ms
image 5/119 /content/drive/MyDrive/yolo_dataset/images/test/1113263349_jpg.rf.ec8d6ab61d0784a2ff03899aa9d0faad.jpg: 960x544 2 Pills, 85.4ms
image 6/119 /content/drive/MyDrive/yolo_dataset/images/test/112161335_jpg.rf.8580c460f32ff302dc0db68a68f52ddd.jpg: 960x544 23 Pills, 14.8ms
image 7/119 /content/drive/MyDrive/yolo_dataset/images/test/1183124474_jpg.rf.93fe7787ed18a3e732daf7162985f191.jpg: 9

In [ ]:
#Result Summary
summary = {
    "config": asdict(config),
    "validation_metrics": results_dict,
    "best_weights": str(trainer.best_weights_path),
    "predictions_count": len(predictions),
    "improvements": {
        "data_augmentation": "Enhanced (Mosaic, Mixup, Copy-Paste, rotation, shear)",
        "optimizer": "AdamW (better for small datasets)",
        "learning_rate": "Reduced with Cosine scheduler",
        "warmup": "3 epochs",
        "patience": "Increased to 50",
        "batch_size": "Reduced to 8 for stability"
    }
}

# Save in JSON
summary_path = Path(config.output_dir) / "training_summary_improved.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

logger.info(f"\nCOMPLETED")
logger.info(f"📄 Summary file is saved at: {summary_path}")

print("\n" + "="*70)
print("🎉 YOLO Pills Detection Completed!")
print("="*70)
print(f"Output directory: {config.output_dir}")
print(f"Best weights: {trainer.best_weights_path}")
print(f"mAP50-95: {results_dict['mAP50-95']:.4f}")
print(f"mAP50: {results_dict['mAP50']:.4f}")
print(f"Precision: {results_dict['Precision']:.4f}")
print(f"Recall: {results_dict['Recall']:.4f}")
print("="*70)


🎉 YOLO Pills Detection Completed!
Output directory: /content/drive/MyDrive/YOLO/Outputs_11 18 ver
Best weights: /content/drive/MyDrive/YOLO/Outputs_11 18 ver/train/weights/best.pt
mAP50-95: 0.6044
mAP50: 0.9535
Precision: 0.9192
Recall: 0.8901
